In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plot
import numpy as np
from numpy import genfromtxt
import csv
import sys, os, re
import pathlib

print(tf.__version__)

2.0.0


In [2]:
# The CANlab/WAVi Pain Study is Pre-Formatted to resemble BIDS neuroimaging formatting
# If your study does not abide to the following structure, please revisit previous scripts
# .../StudyRepo
# -------------> /raw
# -------------------> /*.art
# -------------------> /*.eeg
# -------------------> /*.evt
# -------------> /contigs
# -------------------> /train          2:1 train:test
# -------------------> /test

In [3]:
directory = input("Please give the full path of the contigs folder: ")
os.chdir(directory)

Please give the full path of the contigs folder: /home/clayton/science/CANlab/WAViMedEEG/CANlabStudy/contigs


In [4]:
train_path = pathlib.Path('train/')
test_path = pathlib.Path('test/')

In [5]:
# load in image paths for training set

import random
train_image_paths = list(train_path.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)

train_count = len(train_image_paths)
print("You have", train_count, "training images.")

# load in image paths for testing set?

test_image_paths = list(test_path.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
random.shuffle(test_image_paths)

test_count = len(test_image_paths)
print("You have", test_count, "testing images.")

You have 879 training images.
You have 440 testing images.


In [6]:
#list the available labels
label_names = sorted(item.name for item in train_path.glob('*/') if item.is_dir())
print("Labels discovered:", label_names)

Labels discovered: ['chronic', 'p300']


In [7]:
#assign an index to each label
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print("Label indices:", label_to_index)

Label indices: {'chronic': 0, 'p300': 1}


In [8]:
#create a list of every file and its index label
train_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                     for path in train_image_paths]

test_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in test_image_paths]

In [9]:
def load_numpy_stack(paths):
    numpy_stack = []
    for path in paths:
        numpy_stack.append(genfromtxt(path, delimiter=","))
    return(numpy_stack)

In [10]:
train_arrays = load_numpy_stack(train_image_paths)
test_arrays = load_numpy_stack(test_image_paths)

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_arrays, train_image_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_arrays, test_image_labels))

In [12]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [25]:
def kerasModel(learn, beta1, beta2):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(250, 19)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learn, beta_1=beta1, beta_2=beta2),
                 loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])
    return(model)

In [26]:
def fitModel(model):
    model.fit(train_dataset, epochs=10)
    return(model)

In [27]:
def testModel(model):
    metrics = model.evaluate(test_dataset)
    return(metrics)

In [33]:
rates = np.arange(0.01, 0.1, 0.01)
beta1s = np.arange(0.9, 0.99, 0.01)
beta2s = np.arange(0.99, 0.999, 0.001)

# doing well with default learning rate 0.01 or 0.02
# beta1 does well around 0.91 and 0.92

rate = float(input("What's my learning rate? "))
beta1 = float(input("What's my beta1? "))
beta2 = float(input("What's my beta2? "))

myresults=[]

for beta2 in beta2s:
    compiled = kerasModel(rate, beta1, beta2)
    fitted = fitModel(compiled)
    results = testModel(fitted)
    myresults.append(results)

What's my learning rate? 0.01
What's my beta1? 0.92
What's my beta2? 0
Epoch 1/10
14/14 [==============================] - 0s 27ms/step - loss: 138.1442 - sparse_categorical_accuracy: 0.4778
Epoch 2/10
14/14 [==============================] - 0s 9ms/step - loss: 51.7644 - sparse_categorical_accuracy: 0.6780
Epoch 3/10
14/14 [==============================] - 0s 8ms/step - loss: 19.0331 - sparse_categorical_accuracy: 0.7110
Epoch 4/10
14/14 [==============================] - 0s 8ms/step - loss: 9.9090 - sparse_categorical_accuracy: 0.8237
Epoch 5/10
14/14 [==============================] - 0s 8ms/step - loss: 9.9859 - sparse_categorical_accuracy: 0.7964
Epoch 6/10
14/14 [==============================] - 0s 8ms/step - loss: 9.8897 - sparse_categorical_accuracy: 0.8339
Epoch 7/10
14/14 [==============================] - 0s 8ms/step - loss: 7.2072 - sparse_categorical_accuracy: 0.8578
Epoch 8/10
14/14 [==============================] - 0s 9ms/step - loss: 7.5607 - sparse_categorical_accur

14/14 [==============================] - 0s 9ms/step - loss: 19.2573 - sparse_categorical_accuracy: 0.7440
Epoch 5/10
14/14 [==============================] - 0s 8ms/step - loss: 9.7759 - sparse_categorical_accuracy: 0.7986
Epoch 6/10
14/14 [==============================] - 0s 8ms/step - loss: 3.8949 - sparse_categorical_accuracy: 0.8760
Epoch 7/10
14/14 [==============================] - 0s 8ms/step - loss: 6.1171 - sparse_categorical_accuracy: 0.8294
Epoch 8/10
14/14 [==============================] - 0s 8ms/step - loss: 6.3467 - sparse_categorical_accuracy: 0.8464
Epoch 9/10
14/14 [==============================] - 0s 8ms/step - loss: 8.8350 - sparse_categorical_accuracy: 0.8294
Epoch 10/10
7/7 [==============================] - 0s 14ms/step - loss: 41.1365 - sparse_categorical_accuracy: 0.6159
Epoch 1/10
14/14 [==============================] - 1s 37ms/step - loss: 93.6045 - sparse_categorical_accuracy: 0.4903
Epoch 2/10
14/14 [==============================] - 0s 9ms/step - loss:

In [17]:
print("Loss: ", results[0], "\nAccuracy: ", results[1])

Loss:  0.7553441183907645 
Accuracy:  0.53636366


In [34]:
myresults

[[33.38752801077707, 0.59090906],
 [64.18767765590123, 0.6136364],
 [33.090170179094585, 0.62954545],
 [23.340478352137975, 0.6409091],
 [39.468702043805806, 0.6340909],
 [32.86117826189314, 0.6068182],
 [41.13649232046945, 0.6159091],
 [27.13319287981306, 0.6136364],
 [34.223393304007395, 0.6068182],
 [54.0100473676409, 0.5863636]]

In [30]:
beta1s

array([0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98])